In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

from helpers import count_unique_words, count_unique_ngrams, \
            build_unique_ngrams, create_sentence_vectors, create_sentence_vectors_submission

import sys

import tensorflow as tf
from tensorflow import keras

import gensim   # Not sure whether it is better to use gensim or tensorflow :/
import logging
from gensim.models.phrases import Phrases, Phraser

import multiprocessing

from gensim.models import Word2Vec
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
w2v_model = Word2Vec.load("models/w2v_model_epochs_5_win_5_sg_200.model")

2019-12-14 19:48:09,981 : INFO : loading Word2Vec object from models/w2v_model_epochs_5_win_5_sg_200.model
2019-12-14 19:48:10,667 : INFO : loading wv recursively from models/w2v_model_epochs_5_win_5_sg_200.model.wv.* with mmap=None
2019-12-14 19:48:10,667 : INFO : loading vectors from models/w2v_model_epochs_5_win_5_sg_200.model.wv.vectors.npy with mmap=None
2019-12-14 19:48:10,785 : INFO : setting ignored attribute vectors_norm to None
2019-12-14 19:48:10,786 : INFO : loading vocabulary recursively from models/w2v_model_epochs_5_win_5_sg_200.model.vocabulary.* with mmap=None
2019-12-14 19:48:10,786 : INFO : loading trainables recursively from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.* with mmap=None
2019-12-14 19:48:10,786 : INFO : loading syn1neg from models/w2v_model_epochs_5_win_5_sg_200.model.trainables.syn1neg.npy with mmap=None
2019-12-14 19:48:11,486 : INFO : setting ignored attribute cum_table to None
2019-12-14 19:48:11,487 : INFO : loaded models/w2v_model_epo

In [5]:
len(w2v_model.wv.vocab)

452520

In [6]:
w2v_model.wv.most_similar("car")

2019-12-14 19:48:13,618 : INFO : precomputing L2-norms of word weight vectors


[('minivan', 0.6979140043258667),
 ('dealership', 0.6960715055465698),
 ('towed', 0.6671639680862427),
 ('roundabout', 0.6555763483047485),
 ('prius', 0.6434677839279175),
 ('serviced', 0.643191933631897),
 ('acaliyte', 0.6357315182685852),
 ('swerved', 0.6350514888763428),
 ('1v1', 0.6341693997383118),
 ('mot', 0.6275011301040649)]

In [7]:
word_vector_size = w2v_model.wv.vectors.shape[1]

In [11]:
# This is unfortunately very high memory consumptive

sentence_train_x = np.load("np_arrays/sentences_train_x_0_8.npy")
sentence_train_y = np.load("np_arrays/sentences_train_y_0_8.npy")

sentence_test_x = np.load("np_arrays/sentences_test_x_0_2.npy")
sentence_test_y = np.load("np_arrays/sentences_test_y_0_2.npy")

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint
# now perform training on the new features vectors.

# Build a "deep" neural network with 2 hidden layers. When we see that it somehow works,
# we can start doing some cross validation on it.

for layer_size in range(15, 41, 5):
    
    print("\n\nStarting with number of layers: {}".format(layer_size))
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(layer_size, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                  )

    overfitting_occurrences = 0
    best_accuracy = 0

    filepath="models/weights_2_layers_{}_big".format(layer_size)
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    model.fit(x=sentence_train_x,
              y=sentence_train_y,
              validation_data=(sentence_test_x,  sentence_test_y),
              callbacks=callbacks_list,
              epochs=20,
              use_multiprocessing=True,
              batch_size=128)
        




Starting with number of layers: 15
Train on 2000000 samples, validate on 500000 samples
Epoch 1/20
1996800/2000000 [============================>.] - ETA: 0s - loss: 0.4346 - acc: 0.7904
Epoch 00001: val_acc improved from -inf to 0.79772, saving model to models/weights_2_layers_15_big
2000000/2000000 [==============================] - 29s 14us/sample - loss: 0.4346 - acc: 0.7904 - val_loss: 0.4231 - val_acc: 0.7977
Epoch 2/20
1999488/2000000 [============================>.] - ETA: 0s - loss: 0.4211 - acc: 0.7984
Epoch 00002: val_acc improved from 0.79772 to 0.79917, saving model to models/weights_2_layers_15_big
2000000/2000000 [==============================] - 29s 15us/sample - loss: 0.4211 - acc: 0.7985 - val_loss: 0.4201 - val_acc: 0.7992
Epoch 3/20
1997056/2000000 [============================>.] - ETA: 0s - loss: 0.4175 - acc: 0.8006
Epoch 00003: val_acc improved from 0.79917 to 0.79970, saving model to models/weights_2_layers_15_big
2000000/2000000 [===========================

1997952/2000000 [============================>.] - ETA: 0s - loss: 0.4073 - acc: 0.8063
Epoch 00007: val_acc improved from 0.80374 to 0.80620, saving model to models/weights_2_layers_20_big
2000000/2000000 [==============================] - 27s 14us/sample - loss: 0.4073 - acc: 0.8063 - val_loss: 0.4085 - val_acc: 0.8062
Epoch 8/20
1998592/2000000 [============================>.] - ETA: 0s - loss: 0.4066 - acc: 0.8065
Epoch 00008: val_acc did not improve from 0.80620
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4066 - acc: 0.8065 - val_loss: 0.4088 - val_acc: 0.8055
Epoch 9/20
1998208/2000000 [============================>.] - ETA: 0s - loss: 0.4060 - acc: 0.8069
Epoch 00009: val_acc improved from 0.80620 to 0.80628, saving model to models/weights_2_layers_20_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4060 - acc: 0.8069 - val_loss: 0.4073 - val_acc: 0.8063
Epoch 10/20
1996544/2000000 [============================>.] 

Epoch 13/20
1996800/2000000 [============================>.] - ETA: 0s - loss: 0.4002 - acc: 0.8097
Epoch 00013: val_acc did not improve from 0.80844
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4002 - acc: 0.8097 - val_loss: 0.4032 - val_acc: 0.8082
Epoch 14/20
1998336/2000000 [============================>.] - ETA: 0s - loss: 0.4000 - acc: 0.8099
Epoch 00014: val_acc improved from 0.80844 to 0.80851, saving model to models/weights_2_layers_25_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4000 - acc: 0.8099 - val_loss: 0.4019 - val_acc: 0.8085
Epoch 15/20
1998464/2000000 [============================>.] - ETA: 0s - loss: 0.3995 - acc: 0.8103
Epoch 00015: val_acc did not improve from 0.80851
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.3995 - acc: 0.8103 - val_loss: 0.4037 - val_acc: 0.8079
Epoch 16/20
1998976/2000000 [============================>.] - ETA: 0s - loss: 0.3994 - acc: 0.8102

Epoch 20/20
1997696/2000000 [============================>.] - ETA: 0s - loss: 0.3946 - acc: 0.8128
Epoch 00020: val_acc did not improve from 0.81073
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.3946 - acc: 0.8128 - val_loss: 0.3990 - val_acc: 0.8104


Starting with number of layers: 35
Train on 2000000 samples, validate on 500000 samples
Epoch 1/20
1996416/2000000 [============================>.] - ETA: 0s - loss: 0.4257 - acc: 0.7955
Epoch 00001: val_acc improved from -inf to 0.80266, saving model to models/weights_2_layers_35_big
2000000/2000000 [==============================] - 29s 14us/sample - loss: 0.4256 - acc: 0.7955 - val_loss: 0.4129 - val_acc: 0.8027
Epoch 2/20
1998720/2000000 [============================>.] - ETA: 0s - loss: 0.4105 - acc: 0.8041
Epoch 00002: val_acc improved from 0.80266 to 0.80488, saving model to models/weights_2_layers_35_big
2000000/2000000 [==============================] - 28s 14us/sample - loss: 0.4105 - acc: 0.8041

Epoch 6/20
1999232/2000000 [============================>.] - ETA: 0s - loss: 0.3975 - acc: 0.8112
Epoch 00006: val_acc improved from 0.80888 to 0.80948, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 30s 15us/sample - loss: 0.3975 - acc: 0.8112 - val_loss: 0.3998 - val_acc: 0.8095
Epoch 7/20
1997568/2000000 [============================>.] - ETA: 0s - loss: 0.3963 - acc: 0.8119
Epoch 00007: val_acc improved from 0.80948 to 0.81059, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 39s 19us/sample - loss: 0.3963 - acc: 0.8119 - val_loss: 0.3986 - val_acc: 0.8106
Epoch 8/20
1996160/2000000 [============================>.] - ETA: 0s - loss: 0.3954 - acc: 0.8124
Epoch 00008: val_acc improved from 0.81059 to 0.81066, saving model to models/weights_2_layers_40_big
2000000/2000000 [==============================] - 44s 22us/sample - loss: 0.3954 - acc: 0.8124 - val_loss: 0.3982 - val_acc: 0.810

## To execute this after having trained the above model

After having decided which one is the best model to use, just load the correct model, set the parameters of the model_star so that it matches the one of the best model, and compute the results!

In [55]:
# Recreate exactl the best model
model_star = keras.Sequential([
        keras.layers.InputLayer(input_shape=(sentence_train_x.shape[1],)),   # the input shape is the number of words in the bow dictionary
        keras.layers.Dense(40, activation='relu'),
        keras.layers.Dense(40, activation='relu'),
        keras.layers.Dense(2, activation='softmax')   # Only 0 and 1
    ])

model_star.load_weights('models/weights_2_layers_40_big')

model_star.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [56]:
print(model_star)

In [89]:
# Test it on the test dataset to see if it gives the same best results.
model_star.evaluate(sentence_test_x,  sentence_test_y, verbose=2)[1]
model_star.predict(sentence_test_x)

 - 11s - loss: 0.4045 - acc: 0.8079


array([[4.0681679e-02, 9.5931840e-01],
       [9.9999571e-01, 4.2639813e-06],
       [7.2958276e-02, 9.2704177e-01],
       ...,
       [3.4315959e-01, 6.5684032e-01],
       [6.1612552e-01, 3.8387451e-01],
       [6.4644866e-02, 9.3535513e-01]], dtype=float32)

In [ ]:
df_test

In [97]:
sentence_submission_x = create_sentence_vectors_submission(df_test['sentence'],
                                                           word_vector_size,
                                                           w2v_model)

the number of zero sentences (the sentences which have 0 words in our vocabulary) is 22


In [98]:
model_star.predict(sentence_submission_x)

array([[0.96396494, 0.03603501],
       [0.6632302 , 0.33676985],
       [0.7554848 , 0.24451518],
       ...,
       [0.9986198 , 0.00138015],
       [0.05644046, 0.9435596 ],
       [0.9798126 , 0.02018739]], dtype=float32)

In [101]:
predictions = []
for el in model_star.predict(sentence_submission_x):
    predictions.append(-1 if el[0] > el[1] else 1)

print(predictions[:10])

results = pd.DataFrame({
    "Id": df_test['label'],
    "Prediction": predictions
})

results.head(20)

[-1, -1, -1, 1, -1, -1, -1, 1, 1, 1]


,Id,Prediction
0,1,-1
1,2,-1
2,3,-1
3,4,1
4,5,-1
5,6,-1
6,7,-1
7,8,1
8,9,1
9,10,1


In [102]:
results.to_csv('Submission.csv', index=False)